In [1]:
from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
load_dotenv()

True

In [5]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
from langchain_aws import ChatBedrock
llm_scout = ChatBedrock(
model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# model_id="cohere.command-r-plus-v1:0",
region_name="us-east-1",
model_kwargs={
"temperature": 0.5,
"max_tokens": 2048,
"top_p": 0.9,
}
)


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm_novalite = ChatBedrock(
    model_id="amazon.nova-lite-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

In [7]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("us.meta.llama4-scout-17b-instruct-v1:0",
    model_provider="bedrock",
    region_name="us-east-1")
standard_model = init_chat_model("amazon.nova-lite-v1:0", 
    model_provider="bedrock",
    region_name="us-east-1")


@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

In [8]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

from langchain.agents import create_agent
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)
 
print(response["messages"][-1].content)

Hi there! I'm not able to physically water the office plant, but I can certainly remind everyone to do it. It's important to keep our office environment lively and healthy. If you have a watering schedule or if someone is responsible for it, I can send a reminder via email or our team chat. Let me know how you'd like to proceed!


In [10]:
print(response["messages"][-1].response_metadata["model_name"])

amazon.nova-lite-v1:0


In [11]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)



I'd say in about 3-4 months, once it's outgrown its current pot. I'll keep an eye on the roots and let you know when it's time to transplant it into a larger one.


In [12]:
print(response["messages"][-1].response_metadata["model_name"])

us.meta.llama4-scout-17b-instruct-v1:0
